In [ ]:
# ============================================================
# 10) Save CSVs inside the repo
# ============================================================
OUTPUT_DIR = os.path.join(os.getcwd(), 'bitflip_outputs')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Per-trial CSV
trial_path = os.path.join(OUTPUT_DIR, 'bitflip_per_trial.csv')
df.to_csv(trial_path, index=False)
print(f"Saved per-trial CSV → {trial_path}")

# Aggregated CSV
metric_cols = [c for c in [
    "EditDist","EditDist_Norm","BLEU","METEOR",
    "BERTScore_F1","ROUGE1_F1","ROUGE2_F1","ROUGEL_F1"
] if c in df.columns]
print("Aggregated metrics present:", metric_cols)
if metric_cols:
    summary = df.groupby(
        ["rank","tensor","coord","bit_class","prompt"],
        as_index=False
    ).agg({m: ["mean","median","std"] for m in metric_cols})
    if isinstance(summary.columns, pd.MultiIndex):
        summary.columns = ["_".join(filter(None, c)) for c in summary.columns]
    agg_path = os.path.join(OUTPUT_DIR, 'bitflip_aggregated.csv')
    summary.to_csv(agg_path, index=False)
    print(f"Saved aggregated CSV → {agg_path}")
